# FABRIC Reference Experiment

This notebook has the instructions to run a reference (baseline) experiment on FABRIC testbed to set up the multipath topology below
and then configure the FABRIC nodes to run bulk data transfer between the client and server nodes over one Multipath TCP connection.

![cloudLabtopology](https://github.com/aydini/Multipath-Wireless-Link-Traces/blob/06566873706276b812a8d0c4d4f300a706c6de46/CloudLab%20Reference%20Experiment/cloudlabExperimentTopology.png)

## Set up environment

First, set up your enrivonment in FABRIC.

It is assumed that the keys, the FABRIC config file, and the SSH config file are already created (in the "Configure your environment" notebook) using those default settings (paths, etc).

In [2]:
import os
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/work/fabric_config/slice_key'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/work/fabric_config/slice_key.pub'

os.environ['FABRIC_TOKEN_FILE']=os.environ['HOME']+'/.tokens.json'
os.environ['FABRIC_RC_FILE']=os.environ['HOME']+'/work/fabric_config/fabric_rc'
os.environ['FABRIC_BASTION_SSH_CONFIG_FILE']=os.environ['HOME']+'/work/fabric_config/ssh_config'
os.environ['FABRIC_BASTION_USERNAME']='aydini_0000034746'
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh'

In [14]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

-----------------------------------  -----------------------------------------------
credmgr_host                         cm.fabric-testbed.net
orchestrator_host                    orchestrator.fabric-testbed.net
fabric_token                         /home/fabric/.tokens.json
project_id                           073ee843-2310-45bd-a01f-a15d808827dc
bastion_username                     aydini_0000034746
bastion_key_filename                 /home/fabric/work/.ssh/Aydin_bastion_macbookair
bastion_public_addr                  bastion-1.fabric-testbed.net
bastion_passphrase                   None
slice_public_key_file                /home/fabric/work/fabric_config/slice_key.pub
slice_private_key_file               /home/fabric/work/fabric_config/slice_key
fabric_slice_private_key_passphrase  None
fablib_log_file                      /tmp/fablib/fablib.log
fablib_log_level                     INFO
-----------------------------------  -----------------------------------------------


In [15]:
slice_name="mptcp-base"

## Use previously reserved resources (if needed)

If this raises an error, you don't have a slice for this experiment and need to create one (next section):

In [16]:
if fablib.get_slice(slice_name):
    print("You already have a slice named %s - skip the next part." % slice_name)
    slice = fablib.get_slice(name=slice_name)
    print(slice)

IndexError: list index out of range

## Create slice and reserve resources (only if you don't already have)

See available resources:

In [17]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
MICH         6  180/192  1488/1536  60570/60600    376/381                 2/2                             2/2                            10/10               2/2               3/3
WASH         6  180/192  1500/1536  60398/60600    370/381                 2/2                             2/2                            10/10               2/2               3/3
TACC        10  100/320  1996/2560  114362/116400  553/635                 2/2                             4/4                            16/16               2/4               4/6
UTAH        10  274/320  2376/2560  116170/116400  614/635                 2/2

Create a slice:

In [18]:
# Create a slice
slice = fablib.new_slice(name=slice_name)

Pick a random site and show it:

In [19]:
site_name = fablib.get_random_site()
site_name

'MAX'

Add nodes to the slice (note: use Ubuntu 18 because the MPTCP kernel is based on kernel 4.19):

In [20]:
n_c  = slice.add_node(name="client",    site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')
n_s  = slice.add_node(name="server",    site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')
n_r1 = slice.add_node(name="router1",   site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')
n_e1 = slice.add_node(name="emulator1", site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')
n_r2 = slice.add_node(name="router2",   site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')
n_e2 = slice.add_node(name="emulator2", site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')

Add network interfaces to the nodes:

In [21]:
if_c1     =  n_c.add_component(model="NIC_Basic", name="if_c1").get_interfaces()[0]
if_c2     =  n_c.add_component(model="NIC_Basic", name="if_c2").get_interfaces()[0]
if_s1     =  n_s.add_component(model="NIC_Basic", name="if_s1").get_interfaces()[0]
if_s2     =  n_s.add_component(model="NIC_Basic", name="if_s2").get_interfaces()[0]
if_e1_l   = n_e1.add_component(model="NIC_Basic", name="if_e1_l").get_interfaces()[0]
if_e1_r   = n_e1.add_component(model="NIC_Basic", name="if_e1_r").get_interfaces()[0]
if_r1_l   = n_r1.add_component(model="NIC_Basic", name="if_r1_l").get_interfaces()[0]
if_r1_r   = n_r1.add_component(model="NIC_Basic", name="if_r1_r").get_interfaces()[0]
if_e2_l   = n_e2.add_component(model="NIC_Basic", name="if_e2_l").get_interfaces()[0]
if_e2_r   = n_e2.add_component(model="NIC_Basic", name="if_e2_r").get_interfaces()[0]
if_r2_l   = n_r2.add_component(model="NIC_Basic", name="if_r2_l").get_interfaces()[0]
if_r2_r   = n_r2.add_component(model="NIC_Basic", name="if_r2_r").get_interfaces()[0]

and show them:

In [22]:
print(f"{slice.list_interfaces()}")

Name                  Node       Network      Bandwidth  VLAN    MAC    Physical OS Interface    OS Interface
--------------------  ---------  ---------  -----------  ------  -----  -----------------------  --------------
client-if_c2-p1       client                          0
client-if_c1-p1       client                          0
server-if_s2-p1       server                          0
server-if_s1-p1       server                          0
router1-if_r1_l-p1    router1                         0
router1-if_r1_r-p1    router1                         0
emulator1-if_e1_l-p1  emulator1                       0
emulator1-if_e1_r-p1  emulator1                       0
router2-if_r2_r-p1    router2                         0
router2-if_r2_l-p1    router2                         0
emulator2-if_e2_r-p1  emulator2                       0
emulator2-if_e2_l-p1  emulator2                       0


add Ethernet networks to the slice:

In [23]:
net_1_c_e = slice.add_l2network(name='net_1_c_e', type='L2Bridge', interfaces=[if_c1,    if_e1_l])
net_2_c_e = slice.add_l2network(name='net_2_c_e', type='L2Bridge', interfaces=[if_c2,    if_e2_l])
net_1_e_r = slice.add_l2network(name='net_1_e_r', type='L2Bridge', interfaces=[if_e1_r,  if_r1_l])
net_2_e_r = slice.add_l2network(name='net_2_e_r', type='L2Bridge', interfaces=[if_e2_r,  if_r2_l])
net_1_r_s = slice.add_l2network(name='net_1_r_s', type='L2Bridge', interfaces=[if_r1_r,  if_s1])
net_2_r_s = slice.add_l2network(name='net_2_r_s', type='L2Bridge', interfaces=[if_r2_r,  if_s2])

and see them:

In [24]:
print(f"{slice.list_interfaces()}")

Name                  Node       Network      Bandwidth  VLAN    MAC    Physical OS Interface    OS Interface
--------------------  ---------  ---------  -----------  ------  -----  -----------------------  --------------
client-if_c2-p1       client     net_2_c_e            0
client-if_c1-p1       client     net_1_c_e            0
server-if_s2-p1       server     net_2_r_s            0
server-if_s1-p1       server     net_1_r_s            0
router1-if_r1_l-p1    router1    net_1_e_r            0
router1-if_r1_r-p1    router1    net_1_r_s            0
emulator1-if_e1_l-p1  emulator1  net_1_c_e            0
emulator1-if_e1_r-p1  emulator1  net_1_e_r            0
router2-if_r2_r-p1    router2    net_2_r_s            0
router2-if_r2_l-p1    router2    net_2_e_r            0
emulator2-if_e2_r-p1  emulator2  net_2_e_r            0
emulator2-if_e2_l-p1  emulator2  net_2_c_e            0


Note: Basic NICs claim 0 bandwidth but are 100 Gbps shared by all Basic
NICs on the host.

Submit slice (reserve resources):

Now we are ready to reserve the resources and get the login details.

This step will take a little while, as we communicate with FABRIC to
reserve our requested resources.

-   As it runs, you will see the “State” of each node go from “Ticketed”
    to “Active” (and a “Management IP”, which may be IPv4 or IPv6, will
    be assigned to each one)
-   Then, you’ll wait a few more minutes for some `wait_ssh` and
    `post_boot_config` processes to run.
-   Next, it will show the MAC address and interface name of each of the
    “dataplane” interfaces you set up, as these come up

In [32]:
slice.submit()


-----------  ------------------------------------
Slice Name   mptcp-base
Slice ID     1f567f7c-082b-4bbb-ace1-9c353a260048
Slice State  Closing
Lease End    2022-12-04 22:34:09 +0000
-----------  ------------------------------------

Retry: 0, Time: 21 sec

ID                                    Name       Site    Host    Cores    RAM    Disk    Image              Management IP    State    Error
------------------------------------  ---------  ------  ------  -------  -----  ------  -----------------  ---------------  -------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
f140ae30-5ed0-47c0-aff0-bea5f41f98a2  client     MAX                                     default_ubuntu_18                   Closed   Insufficient resources: No candidates nodes found to serve res: #f140ae30-5ed0-47c0-aff0-bea5f41f98a2 slice: [mptcp-b

'1f567f7c-082b-4bbb-ace1-9c353a260048'

Our final slice status should be “StableOK”:

In [34]:
print(f"{slice}")

-----------  ------------------------------------
Slice Name   mptcp-base
Slice ID     1f567f7c-082b-4bbb-ace1-9c353a260048
Slice State  Closing
Lease End    2022-12-04 22:34:09 +0000
-----------  ------------------------------------


we can block until all hosts are ready to login:

In [27]:
slice.wait_ssh(progress=True)

Waiting for slice . Slice state: Dead


Exception: Slice Exception: Slice Name: mptcp-base, Slice ID: daceb916-a9b4-4297-9bdc-99e345c7a381: Slice Exception: Slice Name: mptcp-base, Slice ID: daceb916-a9b4-4297-9bdc-99e345c7a381: Node: client, Site: MAX, State: Closed, Insufficient resources: No candidates nodes found to serve res: #c5a4d5df-5d64-4da4-a640-af8d549b834e slice: [mptcp-base(daceb916-a9b4-4297-9bdc-99e345c7a381) Owner:aydini@farmingdale.edu] state:[Nascent,Ticketing] 
Slice Exception: Slice Name: mptcp-base, Slice ID: daceb916-a9b4-4297-9bdc-99e345c7a381: Slice Exception: Slice Name: mptcp-base, Slice ID: daceb916-a9b4-4297-9bdc-99e345c7a381: Node: client, Site: MAX, State: Closed, Insufficient resources: No candidates nodes found to serve res: #c5a4d5df-5d64-4da4-a640-af8d549b834e slice: [mptcp-base(daceb916-a9b4-4297-9bdc-99e345c7a381) Owner:aydini@farmingdale.edu] state:[Nascent,Ticketing] 
Node: server, Site: MAX, State: Closed, Insufficient resources: No candidates nodes found to serve res: #b007e917-dde2-42

and we can get login details for every node:

In [ ]:
for node in slice.get_nodes():
    print(f"{node}")

Note: you can open a terminal in the Jupyter environment and use these SSH commands there to log in to the nodes. But, add 

```
 -F ~/work/fabric_config/ssh_config
```

to each SSH command.

Test that resources are available over SSH - should return True:

In [ ]:
slice.test_ssh()

Note: now that the slice is reserved, the variables from the previous section (node and interface variables) are no longer useful. 

### Install MPTCP kernel on client and server

First, we download the packages to the Jupyter workspace (the nodes cannot retrieve them directly from Github because of [the IPv6 issue](https://learn.fabric-testbed.net/knowledge-base/using-ipv4-only-resources-like-github-or-docker-hub-from-ipv6-fabric-sites/).) 

In [ ]:
!wget https://github.com/multipath-tcp/mptcp/releases/download/v0.95.2/linux-headers-4.19.234.mptcp_20220311125841-1_amd64.deb -O /tmp/linux-headers-4.19.234.mptcp_20220311125841-1_amd64.deb
!wget https://github.com/multipath-tcp/mptcp/releases/download/v0.95.2/linux-image-4.19.234.mptcp_20220311125841-1_amd64.deb -O /tmp/linux-image-4.19.234.mptcp_20220311125841-1_amd64.deb
!wget https://github.com/multipath-tcp/mptcp/releases/download/v0.95.2/linux-libc-dev_20220311125841-1_amd64.deb -O /tmp/linux-libc-dev_20220311125841-1_amd64.deb
!wget https://github.com/multipath-tcp/mptcp/releases/download/v0.95.2/linux-mptcp_v0.95.2_20220311125841-1_all.deb -O /tmp/linux-mptcp_v0.95.2_20220311125841-1_all.deb

Then we transfer over to the hosts and install.

In [ ]:
for node_name in ["client", "server"]:
    slice.get_node(name=node_name).upload_file('/tmp/linux-headers-4.19.234.mptcp_20220311125841-1_amd64.deb', '/home/ubuntu/linux-headers-4.19.234.mptcp_20220311125841-1_amd64.deb')
    slice.get_node(name=node_name).upload_file('/tmp/linux-image-4.19.234.mptcp_20220311125841-1_amd64.deb',   '/home/ubuntu/linux-image-4.19.234.mptcp_20220311125841-1_amd64.deb')
    slice.get_node(name=node_name).upload_file('/tmp/linux-libc-dev_20220311125841-1_amd64.deb',               '/home/ubuntu/linux-libc-dev_20220311125841-1_amd64.deb')
    slice.get_node(name=node_name).upload_file('/tmp/linux-mptcp_v0.95.2_20220311125841-1_all.deb',            '/home/ubuntu/linux-mptcp_v0.95.2_20220311125841-1_all.deb')
    slice.get_node(name=node_name).execute("sudo dpkg -i linux*.deb")
    slice.get_node(name=node_name).execute("sudo apt-get install -f")

Restart the nodes:

In [ ]:
for node_name in ["client", "server"]:
    slice.get_node(name=node_name).execute("sudo reboot")

and wait for them to come up again:

In [ ]:
slice.wait_ssh(progress=True)

Check that MPTCP kernel is loaded (`4.19.X.mptcp`):

In [ ]:
for node_name in ["client", "server"]:
    output = slice.get_node(name=node_name).execute("uname -r")[0].strip()
    print(f"Kernel on {node_name}: {output}")
    
    if "mptcp" not in output: # halt notebook execution if kernel is not installed!
        raise Exception ("No MPTCP kernel on %s" % node_name)

### Configure network 

Now we can configure IP addresses on every network interface:

In [ ]:
print(f"{slice.list_interfaces()}")

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

slice.get_node(name='client').ip_addr_add(addr="192.168.10.2", subnet=IPv4Network("192.168.10.0/24"), interface=slice.get_interface(name="client-if_c1-p1"))
slice.get_node(name='client').ip_addr_add(addr="192.168.20.2", subnet=IPv4Network("192.168.20.0/24"), interface=slice.get_interface(name="client-if_c2-p1"))
                                          
slice.get_node(name='server').ip_addr_add(addr="192.168.3.1", subnet=IPv4Network("192.168.3.0/24"), interface=slice.get_interface(name="server-if_s1-p1"))
slice.get_node(name='server').ip_addr_add(addr="192.168.4.1", subnet=IPv4Network("192.168.4.0/24"), interface=slice.get_interface(name="server-if_s2-p1"))

slice.get_node(name='router1').ip_addr_add(addr="192.168.1.2", subnet=IPv4Network("192.168.1.0/24"), interface=slice.get_interface(name="router1-if_r1_l-p1"))
slice.get_node(name='router1').ip_addr_add(addr="192.168.3.2", subnet=IPv4Network("192.168.3.0/24"), interface=slice.get_interface(name="router1-if_r1_r-p1"))
                                          
slice.get_node(name='emulator1').ip_addr_add(addr="192.168.10.1", subnet=IPv4Network("192.168.10.0/24"), interface=slice.get_interface(name="emulator1-if_e1_l-p1"))
slice.get_node(name='emulator1').ip_addr_add(addr="192.168.1.1",  subnet=IPv4Network("192.168.1.0/24"),  interface=slice.get_interface(name="emulator1-if_e1_r-p1"))

slice.get_node(name='router2').ip_addr_add(addr="192.168.2.2", subnet=IPv4Network("192.168.2.0/24"), interface=slice.get_interface(name="router2-if_r2_l-p1"))
slice.get_node(name='router2').ip_addr_add(addr="192.168.4.2", subnet=IPv4Network("192.168.4.0/24"), interface=slice.get_interface(name="router2-if_r2_r-p1"))
                                           
slice.get_node(name='emulator2').ip_addr_add(addr="192.168.20.1", subnet=IPv4Network("192.168.20.0/24"), interface=slice.get_interface(name="emulator2-if_e2_l-p1"))
slice.get_node(name='emulator2').ip_addr_add(addr="192.168.2.1",  subnet=IPv4Network("192.168.2.0/24"),  interface=slice.get_interface(name="emulator2-if_e2_r-p1"))

## Set up multipath networking

### Client and server

Download and install `iproute-mptcp` on client and server:


In [ ]:
!git clone https://github.com/multipath-tcp/iproute-mptcp.git

In [ ]:
for node_name in ["client", "server"]:
    slice.get_node(name=node_name).upload_directory('iproute-mptcp', '/home/ubuntu/')
    slice.get_node(name=node_name).execute("cd /home/ubuntu/iproute-mptcp; sudo make; sudo make install")

Install some other utility software on client and server:

In [ ]:
for node_name in ["client", "server"]:
    slice.get_node(name=node_name).execute("sudo apt update; sudo apt -y install iperf3 net-tools moreutils")

Enable multipath kernel modules on client and server:

In [ ]:
cmds = """
# run sysctl commands to enable mptcp
sudo sysctl -w net.mptcp.mptcp_enabled=1 
sudo sysctl -w net.mptcp.mptcp_checksum=0

# load and configure mptcp congestion control algorithms
sudo modprobe mptcp_balia 
sudo modprobe mptcp_coupled 
sudo modprobe mptcp_olia 
sudo modprobe mptcp_wvegas

# configure mptcp congestion control algorithm to one
sudo sysctl -w net.ipv4.tcp_congestion_control=balia
"""
for node_name in ["client", "server"]:
    slice.get_node(name=node_name).execute(cmds)

Bring up interfaces and add routes on client:

In [ ]:
for iface in slice.get_node(name="client").get_interfaces():
    iface.ip_link_up()
slice.get_node(name="client").ip_route_add(subnet=IPv4Network("192.168.3.0/24"), gateway="192.168.10.1")
slice.get_node(name="client").ip_route_add(subnet=IPv4Network("192.168.4.0/24"), gateway="192.168.20.1")

Enable MPTCP in the experiment interfaces on client:

In [ ]:
for iface in slice.get_node(name="client").get_interfaces():
    slice.get_node(name="client").execute("sudo ip link set dev " + iface.get_os_interface() + " multipath on")

and disable on the control interface:

(note: currently broken, see https://learn.fabric-testbed.net/forums/topic/error-on-get_management_os_interface/
but it doesn't seem to matter)

In [ ]:
control_iface = slice.get_node(name="client").get_management_os_interface()
slice.get_node(name="client").execute("sudo ip link set dev " + control_iface + " multipath off")

Now, add MP routes on client:

In [ ]:
cmds = """
iface1=$(ifconfig | grep -B1 'inet 192.168.10.2' | head -n1 | cut -f1 -d:)
iface2=$(ifconfig | grep -B1 'inet 192.168.20.2' | head -n1 | cut -f1 -d:)

sudo ip rule add from 192.168.10.2 table 1 
sudo ip rule add from 192.168.20.2 table 2 
sudo ip route add 192.168.10.0/24 dev $iface1 scope link table 1
sudo ip route add 192.168.20.0/24 dev $iface2 scope link table 2 
sudo ip route add 192.168.3.0/24 via 192.168.10.1 dev $iface1 table 1 
sudo ip route add 192.168.4.0/24 via 192.168.20.1 dev $iface2 table 2
"""
slice.get_node(name="client").execute(cmds)

Check configuration on client:

In [ ]:
client_ip_ad = slice.get_node(name="client").execute('ip addr')
client_ip_rt = slice.get_node(name="client").execute('ip route')
client_ip_r1 = slice.get_node(name="client").execute('ip route show table 1')
client_ip_r2 = slice.get_node(name="client").execute('ip route show table 2')
for output in [client_ip_ad, client_ip_rt, client_ip_r1, client_ip_r2]:
    print(*output, sep='\n')

Bring up interfaces and add routes on server:

In [ ]:
for iface in slice.get_node(name="server").get_interfaces():
    iface.ip_link_up()
slice.get_node(name="server").ip_route_add(subnet=IPv4Network("192.168.10.0/24"), gateway="192.168.3.2")
slice.get_node(name="server").ip_route_add(subnet=IPv4Network("192.168.20.0/24"), gateway="192.168.4.2")

Check configuration on server:

In [ ]:
server_ip_ad = slice.get_node(name="server").execute('ip addr')
server_ip_rt = slice.get_node(name="server").execute('ip route')
print(*server_ip_ad, sep='\n')
print(*server_ip_rt, sep='\n')

### Gateways

Bring up links:

In [ ]:
for node_name in ["emulator1", "emulator2", "router1", "router2"]:
    for iface in slice.get_node(name=node_name).get_interfaces():
        iface.ip_link_up()

Enable IP forwarding:

In [ ]:
for node_name in ["emulator1", "emulator2", "router1", "router2"]:
    slice.get_node(name=node_name).execute("sudo sysctl -w net.ipv4.ip_forward=1")

Add some routes throughout the network:

In [ ]:
slice.get_node(name="emulator1").ip_route_add(subnet=IPv4Network("192.168.3.0/24"), gateway="192.168.1.2")
slice.get_node(name="emulator2").ip_route_add(subnet=IPv4Network("192.168.4.0/24"), gateway="192.168.2.2")
slice.get_node(name="router1").ip_route_add(subnet=IPv4Network("192.168.10.0/24"), gateway="192.168.1.1")
slice.get_node(name="router2").ip_route_add(subnet=IPv4Network("192.168.20.0/24"), gateway="192.168.2.1")

Add delay at emulator nodes on the "right" side of the network:

In [ ]:
em_iface_r = slice.get_node(name="emulator1").get_interface(name="emulator1-if_e1_r-p1").get_os_interface()
slice.get_node(name="emulator1").execute('sudo tc qdisc replace dev ' + em_iface_r + ' root netem delay 30ms limit 60000')
em_iface_r = slice.get_node(name="emulator2").get_interface(name="emulator2-if_e2_r-p1").get_os_interface()
slice.get_node(name="emulator2").execute('sudo tc qdisc replace dev ' + em_iface_r + ' root netem delay 30ms limit 60000')

Add capacity limit on the router nodes on the "right" side of the network"

In [ ]:
cmds = """
sudo tc qdisc del dev IFACE root # don’t worry if you get RTNETLINK  error
sudo tc qdisc replace dev IFACE root handle 1: htb default 3 
sudo tc class add dev IFACE parent 1: classid 1:3 htb rate 100mbit 
sudo tc qdisc add dev IFACE parent 1:3 handle 3: bfifo limit 375000 
"""

rt_iface_r = slice.get_node(name="router1").get_interface(name="router1-if_r1_r-p1").get_os_interface()
slice.get_node(name="router1").execute(cmds.replace("IFACE", rt_iface_r))
rt_iface_r = slice.get_node(name="router2").get_interface(name="router2-if_r2_r-p1").get_os_interface()
slice.get_node(name="router2").execute(cmds.replace("IFACE", rt_iface_r))

View all the routes:

In [ ]:
for node_name in ["client", "server", "emulator1", "emulator2", "router1", "router2"]: 
    ip_rt = slice.get_node(name=node_name).execute('ip route')
    print("Routes at " + node_name + ":")
    print(*ip_rt, sep='\n')

### Test multipath configuration

Start an iperf3 server on the server node (in daemon mode -D option, so this command won't block).
Then start an iperf3 client on the client node.

In [ ]:
slice.get_node(name="server").execute("iperf3 -s -1 -D")

In [ ]:
iperf_out = slice.get_node(name="client").execute("iperf3 -f m -c 192.168.3.1 -C 'balia' -P 1 -t 20")
print(*iperf_out, sep='\n')

Confirm that you get about 200 Mbps throughput, then you can go ahead and run the experiment to replay the wireless link traces.

## Reference Experiment to Replay the Wireless Link Traces

### Set up experiment

Select Congestion Control algorithm in client and server

In [3]:
#ccAlgos=["balia", "coupled", "olia", "wvegas"] # we can experiment with other CC algos later
ccAlgo="balia"
slice.get_node(name="client").execute("sudo sysctl -w net.mptcp.mptcp_enabled=1")
slice.get_node(name="server").execute("sudo sysctl -w net.mptcp.mptcp_enabled=1")
slice.get_node(name="client").execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + ccAlgo)
slice.get_node(name="server").execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + ccAlgo)

AttributeError: type object 'slice' has no attribute 'get_node'

Copy the scripts needed for the reference experiment and Trace files to router1 and router2

In [4]:
MPTracesRootFolder=os.environ['HOME'] + "/work/Multipath-Wireless-Link-Traces"
nodeHomeFolder="/home/ubuntu"

In [ ]:
for node_name in ["router1", "router2"]:
    slice.get_node(name=node_name).upload_directory(MPTracesRootFolder + "/FABRIC\ Reference\ Experiment", nodeHomeFolder)
    slice.get_node(name=node_name).upload_directory(MPTracesRootFolder + "/Traces", nodeHomeFolder)    

### Execute experiment

In [5]:
print("Starting the Bulk Transfer Reference experiment...") 
# select path and trial
path="7" 
trial="1"

# get the trace pair file names to feed into the routers
wifiTraceFile=nodeHomeFolder + "/Traces/" + path + "_" + trial + "_wifi.csv"  
cellularTraceFile=nodeHomeFolder + "/Traces/"+ path + "_" + trial + "_cellular.csv" 
print("\twith trace files: " + cellularTraceFile + "  " + wifiTraceFile + "  ")

outputDir="FabricReferenceExperimentRawResults"
outputFileName=outputDir + "/" + "rawResults" + "_path" + path + "_trial" + trial + ".txt"
print("\toutput file: " + outputFileName)


Starting the Bulk Transfer Reference experiment...
	with trace files: /home/ubuntu/Traces/7_1_cellular.csv  /home/ubuntu/Traces/7_1_wifi.csv  
	output file: FabricReferenceExperimentRawResults/rawResults_path7_trial1.txt


In [ ]:
# start scripts in router1 and router2 (in non-blocking way using *at* command) 
# so the bandwidths at the routers are regulated based on the collected
# trace pairs (wifi trace on router 1 and cellular trace in router2, respectively)
# important note: make sure to run the script with /bin/bash to make it work with at command
router1Cmd="echo /bin/bash " + nodeHomeFolder + "/FABRIC\ Reference\ Experiment/tputVary.sh " + wifiTraceFile + " | at now"
router2Cmd="echo /bin/bash " + nodeHomeFolder + "/FABRIC\ Reference\ Experiment/tputVary.sh "+ cellularTraceFile +" | at now"

print("\texecuting router commands: " + router1Cmd + " " + router2Cmd)
slice.get_node(name="router1").execute(router1Cmd)
slice.get_node(name="router2").execute(router2Cmd)

print("\trunning the iperf at the server...")
slice.get_node(name="server").execute('iperf3 -s -i 0.1 -D -1') # run as daemon and stop after 1 client connection

clientCmd="iperf3 -f m -c 192.168.3.1 -C '" + ccAlgo + "' -P 1 -i 1 -t 95" + " > " + outputFileName 
print("\tclient iperf command: " + clientCmd)
print("\trunning iperf at the client...") 
slice.get_node(name="client").execute(clientCmd)
#iperf_out = slice.get_node(name="client").execute(clientCmd)
#print(*iperf_out, sep='\n')

print("\tkilling the trace pair feeding scripts at both the routers to end this experiment...")        
killCmd="kill -9 $(ps -ef | grep -i tput | awk '{print $2}')"
slice.get_node(name="router1").execute(killCmd)
slice.get_node(name="router2").execute(killCmd)       
        
print("Finished the Bulk Transfer Reference experiment.")       

### Retrieve data from experiment

Downloading the ran results (output) file from client node to the FABRIC environment.


In [ ]:
slice.get_node(name="client").download_directory('/home/fabric/work/' ,  outputDir )
print("Results downloaded to this folder in this notebook: " + outputDir)

## Delete slice

In [ ]:
#fablib.delete_slice(slice_name)